In [619]:
from IPython.display import clear_output
import numpy as np
import re
import sympy as sp
from sympy import lambdify
from sympy.solvers import solve

In [622]:
hailstone_positions = []
hailstone_velocities = []
with open("./puzzle_inputs/24.txt") as f:
    for line in f:
        x, y, z, vx, vy, vz = map(int, re.findall(r"-?\d+", line))
        hailstone_positions.append((x, y, z))
        hailstone_velocities.append((vx, vy, vz))
hailstone_positions = np.array(hailstone_positions, dtype=np.float128)
hailstone_velocities = np.array(hailstone_velocities, dtype=np.float128)

In [623]:
throw_position = np.array([24, 13, 10], dtype=np.float128)
throw_velocity = np.array([-3, 1, 2], dtype=np.float128)

In [624]:
n = len(hailstone_positions)
n

300

In [472]:
h = sp.IndexedBase("h", shape=(n, 3))
p = sp.IndexedBase("p", shape=(3,))
vh = sp.IndexedBase("v_h", shape=(n, 3))
vp = sp.IndexedBase("v_p", shape=(3,))
t = sp.IndexedBase("t", shape=(n,))
i = sp.Idx("i")

In [473]:
# Generalized distance formula
f = sum((h[i,j] + t[i]*vh[i,j] - p[j] - t[i]*vp[j])**2 for j in range(3))
f = lambdify([h, p, vh, vp, t, i], f)
f(h, p, vh, vp, t, i)

(h[i, 0] - p[0] + t[i]*v_h[i, 0] - t[i]*v_p[0])**2 + (h[i, 1] - p[1] + t[i]*v_h[i, 1] - t[i]*v_p[1])**2 + (h[i, 2] - p[2] + t[i]*v_h[i, 2] - t[i]*v_p[2])**2

In [474]:
# Differentiate and solve for t
df_dt = f(h, p, vh, vp, t, i).diff(t[i])
optimal_t = solve(df_dt, t[i])[0]
optimal_t
# optimize_t = lambdify([h, p, vh, vp, i], solve(df_dt, t[i])[0])

(-h[i, 0]*v_h[i, 0] + h[i, 0]*v_p[0] - h[i, 1]*v_h[i, 1] + h[i, 1]*v_p[1] - h[i, 2]*v_h[i, 2] + h[i, 2]*v_p[2] + p[0]*v_h[i, 0] - p[0]*v_p[0] + p[1]*v_h[i, 1] - p[1]*v_p[1] + p[2]*v_h[i, 2] - p[2]*v_p[2])/(v_h[i, 0]**2 - 2*v_h[i, 0]*v_p[0] + v_h[i, 1]**2 - 2*v_h[i, 1]*v_p[1] + v_h[i, 2]**2 - 2*v_h[i, 2]*v_p[2] + v_p[0]**2 + v_p[1]**2 + v_p[2]**2)

In [475]:
# Substitute the solved t back into the equation
d = sum((h[i,j] - p[j] + optimal_t*(vh[i,j] - vp[j]))**2 for j in range(3))
d = lambdify([h, p, vh, vp, i], d)
d(h, p, vh, vp, i)

((v_h[i, 0] - v_p[0])*(-h[i, 0]*v_h[i, 0] + h[i, 0]*v_p[0] - h[i, 1]*v_h[i, 1] + h[i, 1]*v_p[1] - h[i, 2]*v_h[i, 2] + h[i, 2]*v_p[2] + p[0]*v_h[i, 0] - p[0]*v_p[0] + p[1]*v_h[i, 1] - p[1]*v_p[1] + p[2]*v_h[i, 2] - p[2]*v_p[2])/(v_h[i, 0]**2 - 2*v_h[i, 0]*v_p[0] + v_h[i, 1]**2 - 2*v_h[i, 1]*v_p[1] + v_h[i, 2]**2 - 2*v_h[i, 2]*v_p[2] + v_p[0]**2 + v_p[1]**2 + v_p[2]**2) + h[i, 0] - p[0])**2 + ((v_h[i, 1] - v_p[1])*(-h[i, 0]*v_h[i, 0] + h[i, 0]*v_p[0] - h[i, 1]*v_h[i, 1] + h[i, 1]*v_p[1] - h[i, 2]*v_h[i, 2] + h[i, 2]*v_p[2] + p[0]*v_h[i, 0] - p[0]*v_p[0] + p[1]*v_h[i, 1] - p[1]*v_p[1] + p[2]*v_h[i, 2] - p[2]*v_p[2])/(v_h[i, 0]**2 - 2*v_h[i, 0]*v_p[0] + v_h[i, 1]**2 - 2*v_h[i, 1]*v_p[1] + v_h[i, 2]**2 - 2*v_h[i, 2]*v_p[2] + v_p[0]**2 + v_p[1]**2 + v_p[2]**2) + h[i, 1] - p[1])**2 + ((v_h[i, 2] - v_p[2])*(-h[i, 0]*v_h[i, 0] + h[i, 0]*v_p[0] - h[i, 1]*v_h[i, 1] + h[i, 1]*v_p[1] - h[i, 2]*v_h[i, 2] + h[i, 2]*v_p[2] + p[0]*v_h[i, 0] - p[0]*v_p[0] + p[1]*v_h[i, 1] - p[1]*v_p[1] + p[2]*v_h[i, 2] 

In [681]:
# d(hailstone_positions, throw_position, hailstone_velocities, throw_velocity, ...)

In [462]:
# Compute derivatives with respect to throw position
dd_dp = sp.Array([d(h, p, vh, vp, i).diff(p[index]) for index in range(3)])
dd_dp = lambdify([h, p, vh, vp, i], dd_dp)

# Compute derivatives with respect to throw velocity
dd_dv = sp.Array([d(h, p, vh, vp, i).diff(vp[index]) for index in range(3)])
dd_dv = lambdify([h, p, vh, vp, i], dd_dv)

In [683]:
# dd_dp(hailstone_positions, throw_position, hailstone_velocities, throw_velocity, ...)

In [501]:
dd_dv(hailstone_positions, throw_position, hailstone_velocities, throw_velocity, ...)

array([[-163067.04185997, -163072.59090745, -163067.31128833,
        -163075.51618683, -163075.17437008],
       [ -19769.15446513,  -19766.70707664,  -19760.08959144,
         -19756.81855202,  -19767.55771438],
       [-104591.4369063 , -104593.59124765, -104587.14178545,
        -104590.66589627, -104595.58711278]], dtype=float128)

In [664]:
scale = 1
start = np.float128(200000000000000) / scale
end = np.float128(400000000000000) / scale

In [665]:
throw_position = np.random.uniform(start, end, size=3).astype(np.float128)
throw_velocity = np.random.normal(size=3).astype(np.float128)

throw_position = np.array([461522278379691, 278970483473653, 243127954482398], dtype=np.float128)
throw_velocity = np.array([-336.,   29.,   38.], dtype=np.float128)

In [666]:
print(throw_position)
print(throw_velocity)

[4.61522278e+14 2.78970483e+14 2.43127954e+14]
[-336.   29.   38.]


In [667]:
d(hailstone_positions, throw_position, hailstone_velocities, throw_velocity, ...).mean()

723.85328332885634156

In [673]:
lr = 1e-4
epsilon = 1e-5

step = 0
while abs(0 - (y := d(hailstone_positions, throw_position, hailstone_velocities, throw_velocity, ...).sum())) > epsilon:
    grads = (
        dd_dp(hailstone_positions, throw_position, hailstone_velocities, throw_velocity, ...).mean(1),
        # dd_dv(hailstone_positions, throw_position, hailstone_velocities, throw_velocity, ...).mean(1)
    )
    for v, grad in zip([throw_position, throw_velocity], grads):
        v -= lr*grad
    if step % 100 ==0:
        print("Step:", step)
        print("y = ", y)
        print(list(map(int, throw_position)))
        print(throw_velocity)
        print("Gradients:", grads)
        clear_output(wait=True)
    step += 1

KeyboardInterrupt: 

In [679]:
np.round(throw_position).astype(np.int64).sum()

983620716335751

In [678]:
d(hailstone_positions, np.round(throw_position).astype(np.int64), hailstone_velocities, throw_velocity, ...)

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 9.31322575e-10, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 9.31322575e-10, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      